In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("./HotspotAnalysis/")
import os
from AnalysisFunctions import *
from ROIFunctions import *
from PlottingFunctions import *
import cv2
import numpy as np
from imageio import imwrite
import holoviews as hv
from holoviews import opts
from skimage.measure import block_reduce
from IPython.display import display
hv.notebook_extension('bokeh')

from scipy.stats import skew
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import pingouin as pg

import hotspot_util
fig_config = {'toImageButtonOptions':{'format': 'svg','scale':2}}

***
### Load one GetisOrd Stats CSV, specify `dpath`

In [ ]:
dpath = '/Users/joezaki/Documents/Bentley_Lab/Mesias_2023/Revision_Analyses/Hotspot_Validation_Analysis/Hotspot_Validation_Data/Figure2_ValidationImage/S1_PFC_Adult1b_sectionB4R_20x20/S1_PFC_Adult1b_sectionB4R_20x20_GetisOrdStats.csv'
stats = pd.read_csv(dpath)
stats.head()

### Compute summary stats for z-scores along DV and ML dimensions

In [ ]:
DV_means = np.zeros(len(stats.x.unique()))
DV_sems  = np.zeros(len(stats.x.unique()))
DV_max   = np.zeros(len(stats.x.unique()))
DV_min   = np.zeros(len(stats.x.unique()))
for i, x in enumerate(np.sort(stats.x.unique())):
    row = stats[stats['x'] == x]['Z-Score']
    DV_means[i] = row.mean()
    DV_sems[i]  = row.std() / np.sqrt(len(row))
    DV_max[i]   = row.max()
    DV_min[i]   = row.min()

ML_means = np.zeros(len(stats.y.unique()))
ML_sems  = np.zeros(len(stats.y.unique()))
ML_max   = np.zeros(len(stats.y.unique()))
ML_min   = np.zeros(len(stats.y.unique()))
for i, y in enumerate(np.sort(stats.y.unique())):
    col = stats[stats['y'] == y]['Z-Score']
    ML_means[i] = col.mean()
    ML_sems[i]  = col.std() / np.sqrt(len(col))
    ML_max[i]   = col.max()
    ML_min[i]   = col.min()

zscores = stats.pivot('x','y','Z-Score').values

### Plot mountain plot with mean as line and SEM as error bars

In [ ]:
heatmap_prop = 0.8 # proportion of subplots made up by heatmap
dv_ml_prop = 0.3 # proportion of subplots made up by DV and ML lines
subplot_spacing = 0.05
fig = make_subplots(rows=2, cols=2, row_heights=[heatmap_prop, dv_ml_prop], horizontal_spacing=subplot_spacing, vertical_spacing=subplot_spacing,
                    column_widths=[dv_ml_prop, heatmap_prop])

fig.add_trace(go.Heatmap(z=zscores, colorscale='Spectral_r', colorbar=dict(len=0.7), colorbar_y=0.65), row=1, col=2)

fig.add_trace(go.Scatter(x=np.arange(zscores.shape[0]), y=ML_means, mode='lines+markers', marker=dict(size=5,color='steelblue'), showlegend=False,
                           line=dict(color='steelblue'), error_y=dict(type='data', array=ML_sems, width=0)), row=2, col=2)
fig.add_hline(y=0, row=2, col=2, line_color='black', line_width=1, opacity=1)

fig.add_trace(go.Scatter(x=DV_means, y=np.arange(zscores.shape[0]), mode='lines+markers', marker=dict(size=5, color='darkred'), showlegend=False,
                           line=dict(color='darkred'), error_x=dict(type='data', array=DV_sems, width=0)), row=1, col=1)
fig.add_vline(x=0, row=1, col=1, line_color='black', line_width=1, opacity=1)


fig.update_layout(template='simple_white', width=800, height=800, font=dict(size=18), title_text='Mean +/- SEM')
fig.update_xaxes(visible=False, range=(-1,zscores.shape[1]+1), col=2)
fig.update_yaxes(range=(zscores.shape[0]+1,-1), visible=False, row=1)
fig.update_xaxes(autorange='reversed', dtick=10, row=1, col=1)
fig.update_yaxes(dtick=10, row=2, col=2)
fig.show(config=fig_config)

### Plot mountain plot with mean as line and SEM as ribbons

In [ ]:
heatmap_prop = 0.8 # proportion of subplots made up by heatmap
dv_ml_prop = 0.3 # proportion of subplots made up by DV and ML lines
subplot_spacing = 0.05
fig = make_subplots(rows=2, cols=2, row_heights=[heatmap_prop, dv_ml_prop], horizontal_spacing=subplot_spacing, vertical_spacing=subplot_spacing,
                    column_widths=[dv_ml_prop, heatmap_prop], shared_xaxes=False)

fig.add_trace(go.Heatmap(z=zscores, colorscale='Spectral_r', colorbar=dict(len=0.7), colorbar_y=0.65), row=1, col=2)

fig.add_trace(go.Scattergl(x=np.arange(zscores.shape[1]), y=(ML_means + ML_sems), mode='lines', fill='tonexty',
                           line=dict(color='steelblue', width=0), hoverinfo='skip', showlegend=False, legendgroup='ML'), row=2, col=2)
fig.add_trace(go.Scattergl(x=np.arange(zscores.shape[1]), y=(ML_means - ML_sems), mode='lines', fill='tonexty',
                           line=dict(color='steelblue', width=0), hoverinfo='skip', showlegend=False, legendgroup='ML'), row=2, col=2)
fig.add_trace(go.Scattergl(x=np.arange(zscores.shape[1]), y=ML_means, mode='lines', line=dict(color='steelblue', width=2),
                           hoverinfo='skip', showlegend=False, legendgroup='ML'), row=2, col=2)
fig.add_hline(y=0, row=2, col=2, line_color='black', line_width=1, opacity=1)


fig.add_trace(go.Scattergl(x=(DV_means + DV_sems), y=np.arange(zscores.shape[0]), mode='lines', fill='tonexty',
                           line=dict(color='darkred', width=0), hoverinfo='skip', showlegend=False, legendgroup='DV'), row=1, col=1)
fig.add_trace(go.Scattergl(x=(DV_means - DV_sems), y=np.arange(zscores.shape[0]), mode='lines', fill='tonexty',
                           line=dict(color='darkred', width=0), hoverinfo='skip', showlegend=False, legendgroup='DV'), row=1, col=1)
fig.add_trace(go.Scattergl(x=DV_means, y=np.arange(zscores.shape[0]), mode='lines', line=dict(color='darkred', width=2),
                           hoverinfo='skip', showlegend=False, legendgroup='DV'), row=1, col=1)
fig.add_vline(x=0, row=1, col=1, line_color='black', line_width=1, opacity=1)


fig.update_layout(template='simple_white', width=800, height=800, font=dict(size=18), title_text='Mean +/- SEM')
fig.update_xaxes(visible=False, range=(-1,zscores.shape[1]+1), col=2)
fig.update_yaxes(range=(zscores.shape[0]+1,-1), visible=False, row=1)
fig.update_xaxes(autorange='reversed', dtick=10, row=1, col=1)
fig.update_yaxes(dtick=10, row=2, col=2)
fig.show(config=fig_config)

### Plot mountain plot with mean as line and max/min as ribbons

In [ ]:
heatmap_prop = 0.8 # proportion of subplots made up by heatmap
dv_ml_prop = 0.3 # proportion of subplots made up by DV and ML lines
subplot_spacing = 0.05
fig = make_subplots(rows=2, cols=2, row_heights=[heatmap_prop, dv_ml_prop], horizontal_spacing=subplot_spacing, vertical_spacing=subplot_spacing,
                    column_widths=[dv_ml_prop, heatmap_prop], shared_xaxes=False)

fig.add_trace(go.Heatmap(z=zscores, colorscale='Spectral_r', colorbar=dict(len=0.7), colorbar_y=0.65), row=1, col=2)

fig.add_trace(go.Scattergl(x=np.arange(zscores.shape[1]), y=ML_max, mode='lines', fill='tonexty',
                           line=dict(color='steelblue', width=0), hoverinfo='skip', showlegend=False, legendgroup='ML'), row=2, col=2)
fig.add_trace(go.Scattergl(x=np.arange(zscores.shape[1]), y=ML_min, mode='lines', fill='tonexty',
                           line=dict(color='steelblue', width=0), hoverinfo='skip', showlegend=False, legendgroup='ML'), row=2, col=2)
fig.add_trace(go.Scattergl(x=np.arange(zscores.shape[1]), y=ML_means, mode='lines', line=dict(color='steelblue', width=2),
                           hoverinfo='skip', showlegend=False, legendgroup='ML'), row=2, col=2)
fig.add_hline(y=0, row=2, col=2, line_color='black', line_width=1, opacity=1)


fig.add_trace(go.Scattergl(x=DV_max, y=np.arange(zscores.shape[0]), mode='lines', fill='tonexty',
                           line=dict(color='darkred', width=0), hoverinfo='skip', showlegend=False, legendgroup='DV'), row=1, col=1)
fig.add_trace(go.Scattergl(x=DV_min, y=np.arange(zscores.shape[0]), mode='lines', fill='tonexty',
                           line=dict(color='darkred', width=0), hoverinfo='skip', showlegend=False, legendgroup='DV'), row=1, col=1)
fig.add_trace(go.Scattergl(x=DV_means, y=np.arange(zscores.shape[0]), mode='lines', line=dict(color='darkred', width=2),
                           hoverinfo='skip', showlegend=False, legendgroup='DV'), row=1, col=1)
fig.add_vline(x=0, row=1, col=1, line_color='black', line_width=1, opacity=1)

fig.add_shape(type='rect', x0=0, x1=1, y0=0, y1=1)


fig.update_layout(template='simple_white', width=800, height=800, font=dict(size=18), title_text='Mean +/- Max/Min')
fig.update_xaxes(visible=False, range=(-1,zscores.shape[1]+1), col=2)
fig.update_yaxes(range=(zscores.shape[0]+1,-1), visible=False, row=1)
fig.update_xaxes(autorange='reversed', dtick=20, row=1, col=1)
fig.update_yaxes(dtick=20, row=2, col=2)
fig.show(config=fig_config)

### Plot mountain plot with mean as line and SEM as ribbons and max/min as dotted lines

In [ ]:
heatmap_prop = 0.8 # proportion of subplots made up by heatmap
dv_ml_prop = 0.3 # proportion of subplots made up by DV and ML lines
subplot_spacing = 0.05
fig = make_subplots(rows=2, cols=2, row_heights=[heatmap_prop, dv_ml_prop], horizontal_spacing=subplot_spacing, vertical_spacing=subplot_spacing,
                    column_widths=[dv_ml_prop, heatmap_prop], shared_xaxes=False)

fig.add_trace(go.Heatmap(z=zscores, colorscale='Spectral_r', colorbar=dict(len=0.7), colorbar_y=0.65), row=1, col=2)

fig.add_trace(go.Scattergl(x=np.arange(zscores.shape[1]), y=(ML_means + ML_sems), mode='lines', fill='tonexty',
                           line=dict(color='steelblue', width=0), hoverinfo='skip', showlegend=False, legendgroup='ML'), row=2, col=2)
fig.add_trace(go.Scattergl(x=np.arange(zscores.shape[1]), y=(ML_means - ML_sems), mode='lines', fill='tonexty',
                           line=dict(color='steelblue', width=0), hoverinfo='skip', showlegend=False, legendgroup='ML'), row=2, col=2)
fig.add_trace(go.Scattergl(x=np.arange(zscores.shape[1]), y=ML_means, mode='lines', line=dict(color='steelblue', width=2),
                           hoverinfo='skip', showlegend=False, legendgroup='ML'), row=2, col=2)
fig.add_hline(y=0, row=2, col=2, line_color='black', line_width=1, opacity=1)

fig.add_trace(go.Scattergl(x=np.arange(zscores.shape[1]), y=ML_max, mode='lines', fill='tonexty',
                           line=dict(color='steelblue', width=2, dash='dot'), hoverinfo='skip', showlegend=False, legendgroup='ML'), row=2, col=2)
fig.add_trace(go.Scattergl(x=np.arange(zscores.shape[1]), y=ML_min, mode='lines', fill='tonexty',
                           line=dict(color='steelblue', width=2, dash='dot'), hoverinfo='skip', showlegend=False, legendgroup='ML'), row=2, col=2)


fig.add_trace(go.Scattergl(x=(DV_means + DV_sems), y=np.arange(zscores.shape[0]), mode='lines', fill='tonexty',
                           line=dict(color='darkred', width=0), hoverinfo='skip', showlegend=False, legendgroup='DV'), row=1, col=1)
fig.add_trace(go.Scattergl(x=(DV_means - DV_sems), y=np.arange(zscores.shape[0]), mode='lines', fill='tonexty',
                           line=dict(color='darkred', width=0), hoverinfo='skip', showlegend=False, legendgroup='DV'), row=1, col=1)
fig.add_trace(go.Scattergl(x=DV_means, y=np.arange(zscores.shape[0]), mode='lines', line=dict(color='darkred', width=2),
                           hoverinfo='skip', showlegend=False, legendgroup='DV'), row=1, col=1)
fig.add_vline(x=0, row=1, col=1, line_color='black', line_width=1, opacity=1)

fig.add_trace(go.Scattergl(x=DV_max, y=np.arange(zscores.shape[0]), mode='lines', fill='tonexty',
                           line=dict(color='darkred', width=2, dash='dot'), hoverinfo='skip', showlegend=False, legendgroup='DV'), row=1, col=1)
fig.add_trace(go.Scattergl(x=DV_min, y=np.arange(zscores.shape[0]), mode='lines', fill='tonexty',
                           line=dict(color='darkred', width=2, dash='dot'), hoverinfo='skip', showlegend=False, legendgroup='DV'), row=1, col=1)


fig.update_layout(template='simple_white', width=800, height=800, font=dict(size=18), title_text='Mean +/- SEM and Max/Min')
fig.update_xaxes(visible=False, range=(-1,zscores.shape[1]+1), col=2)
fig.update_yaxes(range=(zscores.shape[0]+1,-1), visible=False, row=1)
fig.update_xaxes(autorange='reversed', dtick=20, row=1, col=1)
fig.update_yaxes(dtick=20, row=2, col=2)
fig.show(config=fig_config)